In [9]:
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import pprint
import math
import random

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.feature_extraction.text import CountVectorizer

from joblib import dump, load
import pickle

from utils.preprocess_data import remove_time_stamp

#from utils.queryDB import queryDBForTestOutput
# from preprocessData import generateFeatureWordsList, generateLabelsIndex, \
#                             preprocessDBData, shuffleDataLabelPairs, \
#                             padding_processed_data, preprocessTFIDFData


In [10]:
train_data_path = 'data/GitHubData/IssueContent'
train_data_extra_path = 'data/GitHubData/IssueContentQuote'
train_data = []
train_label = []

train_files = os.listdir(train_data_path)
train_files_extra = os.listdir(train_data_extra_path)

for train_file in train_files:
    if (train_file.endswith('txt')):
        cur_train_label = train_file.strip(".txt")
        train_label.append(cur_train_label)

        cur_train_data = ""
        with open(os.path.join(train_data_path, train_file)) as f1:
            cur_train_data += f1.read()
        # Attach related extra quotes file data if exists
        possible_quotes_filename = cur_train_label + "_quotes.txt"
        possible_quotes_file_path = os.path.join(train_data_extra_path, possible_quotes_filename)
        if os.path.isfile(possible_quotes_file_path):
            with open(possible_quotes_file_path) as f2:
                cur_train_data += f2.read()
        train_data.append(cur_train_data)

In [ ]:
df_git = pd.DataFrame([train_data, train_label]).T
df_git.columns= ['git_issue_content', 'labels']
df_git.head()

In [ ]:
# pre-process git data
df_git['git_issue_content'] = [remove_time_stamp(content) for content in df_git['git_issue_content']]
df_git.head()

In [ ]:
test_data_path = 'data/JenkinsData'
test_data = []
test_label = []

test_files = os.listdir(test_data_path)
for test_file in test_files:
    if (test_file.endswith('txt')):
        # test_label needs to remove .txt and trailing links (_http...) in file name, e.g. from openj9_1.txt and openj9_1_httpxxx.txt
        cur_test_label = test_file.strip(".txt").split("_http")[0]
        test_label.append(cur_test_label)
        with open(os.path.join(test_data_path, test_file)) as f:
            test_data.append(f.read())

df_jenkins = pd.DataFrame([test_data, test_label]).T
df_jenkins.columns= ['jenkins_content', 'labels']
df_jenkins.head()


In [ ]:
# pre-process jenkins data
df_jenkins['jenkins_content'] = [remove_time_stamp(content) for content in df_jenkins['jenkins_content']]
df_jenkins.head()

In [ ]:
# add train test matching in df

In [ ]:
# df_git['clean_text'] = df_git['git_issue_content'].apply(process_text)
# df_git.head()

In [ ]:
clean_df_git = df_git.copy()

def create_label_dict(labels, label_dict_name2num, label_dict_num2name):
    index = 0
    for label in labels:
        if label not in label_dict_name2num:
            label_dict_num2name[index] = label.rstrip('.txt')
            label_dict_name2num[label] = index
            index += 1

labels_git_name = clean_df_git.pop("labels") 
label_dict_name2num = dict()
label_dict_num2name = dict()
create_label_dict(labels_git_name, label_dict_name2num, label_dict_num2name)
labels_git_num = [label_dict_name2num[x] for x in labels_git_name]

In [ ]:
# save num2name for TRSS use
try: 
    os.makedirs('data/TempData', exist_ok=True)
    num2name_file = open('data/TempData/num2name_file', 'wb')
    pickle.dump(label_dict_num2name, num2name_file)
    num2name_file.close() 
  
except:
    print("Failed to save num2name_file")

In [ ]:
clean_df_git.head()

In [ ]:
# tfidf can add more parameter settings
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer_train_vectors = tfidf_vectorizer.fit_transform(clean_df_git.pop('git_issue_content'))


# save vectorizer for TRSS use
try: 
    vectorizer_file = open('data/TempData/vectorizer_file.pk', 'wb')
    pickle.dump(tfidf_vectorizer, vectorizer_file)
    vectorizer_file.close()
except: 
    print("Failed to save vectorizer_file")


train_df = pd.DataFrame.sparse.from_spmatrix(tfidf_vectorizer_train_vectors, columns=tfidf_vectorizer.get_feature_names())
print("train data after tfidf: \n", train_df.head())


In [ ]:
# add val when more data available
val_df = train_df

# test df is from jenkins
clean_df_jenkins = df_jenkins.copy()
labels_jenkins_name = clean_df_jenkins.pop("labels")
labels_jenkins_num = [label_dict_name2num[x] for x in labels_jenkins_name]
print("test labels list: ", labels_jenkins_num)
test_df = pd.DataFrame.sparse.from_spmatrix(tfidf_vectorizer.transform(clean_df_jenkins.pop('jenkins_content')), columns=tfidf_vectorizer.get_feature_names())
print("test data after tfidf: \n", test_df.head())

In [ ]:
train_labels = np.array(labels_git_num)
# add val when more data available# add val when more data available
val_labels = np.array(labels_git_num)
test_labels = np.array(labels_jenkins_num)

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [ ]:
# Normalize the input features using the sklearn StandardScaler. This will set the mean to 0 and standard deviation to 1.
standardScaler = StandardScaler()
train_features = standardScaler.fit_transform(train_features)
val_features = standardScaler.transform(val_features)
test_features = standardScaler.transform(test_features)


# save standardScaler for TRSS use
try: 
    standardScaler_file = open('data/TempData/standardScaler_file.pk', 'wb')
    pickle.dump(standardScaler, standardScaler_file)
    standardScaler_file.close()
except: 
    print("Failed to save standardScaler_file")

print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

In [ ]:
# Preliminary models evaluation with default parameters

# Creating a dict of the several models
# Temporarily use only two models to reduce training time
model_dict = {'Decsision Tree Model': DecisionTreeClassifier(random_state=3),
              # 'Dummy Model' : DummyClassifier(random_state=3),
              # 'Stochastic Gradient Descent Model' : SGDClassifier(random_state=3, loss='log'),
              # 'AdaBoost Model': AdaBoostClassifier(random_state=3),
              # 'Gaussian Naive Bayes Model': GaussianNB(),
              # 'K Nearest Neighbor Model': KNeighborsClassifier(),
              'Random Forest Model': RandomForestClassifier(random_state=3)}

# Train test split with stratified sampling for evaluation
# X_train, X_test, y_train, y_test = train_features, val_features, train_labels, val_labels
X_train, X_test, y_train, y_test = train_features, test_features, train_labels, test_labels
# X_train, X_test, y_train, y_test = train_test_split(X, 
#                                                     y, 
#                                                     test_size = .3, 
#                                                     shuffle = True, 
#                                                     stratify = y, 
#                                                     random_state = 3)

#Function to get the scores for each model in a df
def model_scores_df(model_dict):   
    model_names, ac_scores_list, p_scores_list, r_scores_list, f1_scores_list = [], [], [], [], []
    for cur_model_name,cur_model in model_dict.items():   
        model_names.append(cur_model_name)
        cur_model.fit(X_train, y_train)
        label_pred = cur_model.predict(X_test)
        ac_scores_list.append(accuracy_score(y_test, label_pred))
        p_scores_list.append(precision_score(y_test, label_pred, average='macro'))
        r_scores_list.append(recall_score(y_test, label_pred, average='macro'))
        f1_scores_list.append(f1_score(y_test, label_pred, average='macro'))
    
    model_comparison_df = pd.DataFrame([model_names, ac_scores_list, p_scores_list, r_scores_list, f1_scores_list]).T
    model_comparison_df.columns = ['model_names', 'accuracy_scores', 'precision_scores', 'recall_scores', 'f1_scores']
        
    model_comparison_df = model_comparison_df.sort_values(by='f1_scores', ascending=False).reset_index(drop=True)
    return model_comparison_df

scores_df = model_scores_df(model_dict)
scores_df

In [ ]:
# Saving and loading the best model

best_model_name = scores_df["model_names"][0]
print("Best model found:", best_model_name)
best_model = model_dict[best_model_name]

#Save the best model
dump(best_model, 'MLModel.joblib')

#Load the best model
saved_best_model = load('MLModel.joblib')

#Predict with the best model
saved_best_model_predictions = saved_best_model.predict(test_features)
saved_best_model_predictions_proba = saved_best_model.predict_proba(test_features)

print("Saved_Best_Model Sample 0 predicted label:", saved_best_model_predictions[0])
print("Saved_Best_Model Sample 0 prediction probability:", saved_best_model_predictions_proba[0])